In [6]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.tri as mtri

# Physical constants
L, W = 1.0, 0.5    # domain size (cm)
lambda_, mu = 5e9, 5e9  # elastic constants (Pa)
h = 1.0            # thickness (cm)
sf = 1e9           # stress scaling

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        # now outputs: u_x, u_y, E_xx, E_yy, E_xy
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 256),
            nn.Softplus(beta=10),
            nn.Linear(256, 256),
            nn.Softplus(beta=10),
            nn.Linear(256, 256),
            nn.Softplus(beta=10),
            nn.Linear(256, 6)
        )

    def forward(self, x, y):
        # normalize into [-1,1]
        xi  = 2.0 * x / L
        eta = 2.0 * y / W

        raw = self.net(torch.cat([xi, eta], dim=1))
        return raw

# compute true displacement gradients and small‐strain tensor via autograd
def strain_tensor(u_net, v_net, x, y):
    # ∂u/∂x, ∂u/∂y
    u_x_true = torch.autograd.grad(u_net, x,
                    grad_outputs=torch.ones_like(u_net),
                    retain_graph=True, create_graph=True)[0]
    u_y_true = torch.autograd.grad(u_net, y,
                    grad_outputs=torch.ones_like(u_net),
                    retain_graph=True, create_graph=True)[0]
    # ∂v/∂x, ∂v/∂y
    v_x_true = torch.autograd.grad(v_net, x,
                    grad_outputs=torch.ones_like(v_net),
                    retain_graph=True, create_graph=True)[0]
    v_y_true = torch.autograd.grad(v_net, y,
                    grad_outputs=torch.ones_like(v_net),
                    retain_graph=True, create_graph=True)[0]

    # small‐strain components
    Exx = u_x_true
    Eyy = v_y_true
    Exy = 0.5 * (u_y_true + v_x_true)

    return u_x_true, u_y_true, v_x_true, v_y_true, Exx, Eyy, Exy


# stress from strain (unchanged)
def stress_tensor(Exx, Eyy, Exy):
    TrE = Exx + Eyy
    Sxx = h * ((lambda_/sf) * TrE + 2 * (mu/sf) * Exx)
    Syy = h * ((lambda_/sf) * TrE + 2 * (mu/sf) * Eyy)
    Sxy = h * (2 * (mu/sf) * Exy)
    return Sxx, Syy, Sxy


# FO‐PINN physics loss with six‐output network: [u, v, u_x, u_y, v_x, v_y]
def physics_loss(model, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)

    out = model(x, y)
    # unpack network outputs
    u_net, v_net, u_x_net, u_y_net, v_x_net, v_y_net = (
        out[:,  i:i+1] for i in range(6)
    )

    # 1) true gradients & strains via autograd
    u_x_true, u_y_true, v_x_true, v_y_true, \
      Exx_true, Eyy_true, Exy_true = strain_tensor(u_net, v_net, x, y)

    # 2) compatibility loss
    w_comp = 4.0
    loss_grad = (
        torch.mean((u_x_net - u_x_true)**2) +
        torch.mean((u_y_net - u_y_true)**2) +
        torch.mean((v_x_net - v_x_true)**2) +
        torch.mean((v_y_net - v_y_true)**2)
    )

    lc = (loss_grad) * w_comp
    
    Exx_net, Eyy_net = u_x_net, v_y_net
    Exy_net = 0.5 * (u_y_net + v_x_net)
    Sxx, Syy, Sxy = stress_tensor(Exx_net, Eyy_net, Exy_net)

    # 4) first‐order PDE residuals
    Sxx_x = torch.autograd.grad(Sxx, x,
                 grad_outputs=torch.ones_like(Sxx),
                 retain_graph=True, create_graph=True)[0]
    Sxy_y = torch.autograd.grad(Sxy, y,
                 grad_outputs=torch.ones_like(Sxy),
                 retain_graph=True, create_graph=True)[0]
    Syy_y = torch.autograd.grad(Syy, y,
                 grad_outputs=torch.ones_like(Syy),
                 retain_graph=True, create_graph=True)[0]
    Sxy_x = torch.autograd.grad(Sxy, x,
                 grad_outputs=torch.ones_like(Sxy),
                 retain_graph=True, create_graph=True)[0]

    rx = Sxx_x + Sxy_y
    ry = Syy_y + Sxy_x
    lpde = torch.mean(rx**2 + ry**2)

    return lpde + lc


# BC loss
def boundary_condition_loss(model, L, W):
    w_A, w_D, w_C, w_B = 1.0, 1.0, 1.0, 1.0 # Weight for each 
                                            # side of the boundary condition
    
    # A: x = -L/2, u=v=0
    y_A = torch.linspace(-W/2, W/2, 500).reshape(-1,1)
    x_A = -L/2 * torch.ones_like(y_A)
    out_A = model(x_A, y_A)
    u_A, v_A = out_A[:,0:1], out_A[:,1:2]
    loss_A = torch.mean(u_A**2 + v_A**2)
    
    # D: x = +L/2, u_x = 0.025L, u_y = 0
    x_D = L/2 * torch.ones_like(y_A)
    out_D = model(x_D, y_A)
    u_D, v_D = out_D[:,2:3], out_D[:,3:4]
    loss_D = torch.mean((u_D - 0.02*L)**2 + v_D**2)
    
    # C: y = -W/2 traction-free -> sigma_yy=0, sigma_xy=0
    x_C = torch.linspace(-L/2, L/2, 200).reshape(-1,1)
    y_C = -W/2 * torch.ones_like(x_C)
    out_C = model(x_C, y_C)
    Exx_C, Eyy_C = out_C[:,2:3], out_C[:,5:6]
    Exy_C = 0.5*(out_C[:,3:4] + out_C[:,4:5])
    _, Syy_C, Sxy_C = stress_tensor(Exx_C, Eyy_C, Exy_C)
    loss_C = torch.mean(Syy_C**2 + Sxy_C**2)
    
    # B: y = +W/2 traction-free -> sigma_yy=0, sigma_xy=0
    y_B = W/2 * torch.ones_like(x_C)
    out_B = model(x_C, y_B)
    Exx_B, Eyy_B = out_B[:,2:3], out_B[:,5:6]
    Exy_B = 0.5*(out_B[:,3:4] + out_B[:,4:5])
    _, Syy_B, Sxy_B = stress_tensor(Exx_B, Eyy_B, Exy_B)
    loss_B = torch.mean(Syy_B**2 + Sxy_B**2)
    return w_A*loss_A + w_D*loss_D + w_C*loss_C + w_B*loss_B


# train 
def train_pinn(model, optimizer, n_epochs, n_points, L, W):
    history = []
    w_pde, w_bc = 1.0, 50.0

    for ep in range(n_epochs):
        x = torch.rand(n_points,1)*L - L/2
        y = torch.rand(n_points,1)*W - W/2

        lp = physics_loss(model, x, y)           
        lb = boundary_condition_loss(model, L, W)  
        loss = w_pde*lp + w_bc*lb

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        history.append(loss.item())
        if ep % 500 == 0:
            print(f"Epoch {ep:5d} │ Total: {loss.item():.3e} │ PDE+comp: {lp.item():.3e} │ BC: {lb.item():.3e}")
    return history

# initialize and run
model     = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 10000
n_points = 1000
loss_history = train_pinn(model, optimizer, n_epochs, n_points, L, W)

torch.save(model.state_dict(), "pinn_elasticity.pth")
print("✅ Model weights saved to pinn_elasticity.pth")


Epoch     0 │ Total: 4.786e+02 │ PDE+comp: 2.096e-01 │ BC: 9.567e+00
Epoch   500 │ Total: 6.346e-04 │ PDE+comp: 5.193e-04 │ BC: 2.305e-06
Epoch  1000 │ Total: 2.650e-04 │ PDE+comp: 2.028e-04 │ BC: 1.245e-06
Epoch  1500 │ Total: 1.625e-04 │ PDE+comp: 1.171e-04 │ BC: 9.089e-07
Epoch  2000 │ Total: 1.109e-04 │ PDE+comp: 7.678e-05 │ BC: 6.816e-07
Epoch  2500 │ Total: 8.542e-05 │ PDE+comp: 5.996e-05 │ BC: 5.091e-07
Epoch  3000 │ Total: 6.257e-05 │ PDE+comp: 4.171e-05 │ BC: 4.172e-07
Epoch  3500 │ Total: 3.256e-04 │ PDE+comp: 3.682e-05 │ BC: 5.775e-06
Epoch  4000 │ Total: 2.701e-01 │ PDE+comp: 2.058e-04 │ BC: 5.397e-03
Epoch  4500 │ Total: 6.634e-05 │ PDE+comp: 3.499e-05 │ BC: 6.270e-07
Epoch  5000 │ Total: 5.233e-05 │ PDE+comp: 2.386e-05 │ BC: 5.693e-07
Epoch  5500 │ Total: 1.972e-04 │ PDE+comp: 2.147e-05 │ BC: 3.514e-06
Epoch  6000 │ Total: 8.308e-04 │ PDE+comp: 2.332e-05 │ BC: 1.615e-05
Epoch  6500 │ Total: 1.227e-04 │ PDE+comp: 1.979e-05 │ BC: 2.059e-06
Epoch  7000 │ Total: 1.539e-03 │ P